# Segmentation 3D

## 0. Imports

In [ ]:
import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display.notebook as phm_display_notebook
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.segmentation as phm_seg

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [ ]:
plant_number = 2
voxels_size = 16

bin_images = phm_data.bin_images("data/plant_2")
calibrations = phm_data.calibrations("data/plant_2")
voxel_grid = phm_data.voxel_grid("data", plant_number=2, voxels_size=voxels_size)

### 1.2 Display voxel grid

In [ ]:
phm_display_notebook.show_voxel_grid(voxel_grid, size=1)

## 2. Skeletonization

### 2.1 Compute Graph and the skeleton
Compute the skeleton of the plant voxelgrid

In [ ]:
graph = phm_seg.graph_from_voxel_grid(voxel_grid, connect_all_point=True)
voxel_skeleton = phm_seg.skeletonize(voxel_grid, graph)

### 2.2 Display it

In [ ]:
phm_display_notebook.show_skeleton(voxel_skeleton, with_voxel=True, size=1)

### 2.3 Remove not enouth visible segment

In [ ]:
# Select images
image_projection = list()
for angle in bin_images["side"]:
    projection = calibrations["side"].get_projection(angle)
    image_projection.append((bin_images["side"][angle], projection))

voxel_skeleton_denoised = phm_seg.segment_reduction(
    voxel_skeleton, image_projection, required_visible=5, nb_min_pixel=400
)

### 2.4 Display it

In [ ]:
phm_display_notebook.show_skeleton(voxel_skeleton_denoised, size=1)

## 3. Maize Segmentation

### 3.1 Process

Voxel skeleton are segmented in several label "stem", "mature_leaf", "growing_leaf" and "unknow"

In [ ]:
vms = phm_seg.maize_segmentation(voxel_skeleton_denoised, graph)

### 3.2 Write / Read

In [ ]:
# Write
vms.write_to_json_gz("voxel_maize_segmentation.json")

In [ ]:
# Load
vms = phm_obj.VoxelSegmentation.read_from_json_gz("voxel_maize_segmentation.json")

### 3.3 Display

In [ ]:
phm_display_notebook.show_segmentation(vms, size=1)

## 4. Maize Analysis

### 4.1 Process

In [ ]:
vmsi = phm_seg.maize_analysis(vms)

### 4.2 How to save analysis result :

In [ ]:
vmsi.write_to_json_gz("voxel_maize_segmentation_info.json.gz")

In [ ]:
vmsi = phm_obj.VoxelSegmentation.read_from_json_gz(
    "voxel_maize_segmentation_info.json.gz"
)

### 4.3 Visualize results: 

In [ ]:
phm_display_notebook.show_segmentation(vmsi, size=1)